![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FMLOps&file=Vertex+AI+Pipelines+-+Notifications.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/MLOps/Vertex%20AI%20Pipelines%20-%20Notifications.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FMLOps%2FVertex%2520AI%2520Pipelines%2520-%2520Notifications.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/MLOps/Vertex%20AI%20Pipelines%20-%20Notifications.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/MLOps/Vertex%20AI%20Pipelines%20-%20Notifications.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Vertex AI Pipelines - Notifications

TEMPLATE IN PROGRESS

How to send email notifications based on pipelines execution.
- pre-built component to send email notificaitons on completion - any status
- how to gather pipeline stastus on exist
- custom built component to send email conditional on pipelines status at exist
    - complete with errors

---
## Colab Setup

To run this notebook in Colab run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
    print('Colab authorized to GCP')
except Exception:
    print('Not a Colab Environment')
    pass

Not a Colab Environment


---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform', '1.51.0'),
    ('google_cloud_pipeline_components', 'google-cloud-pipeline-components'),
    ('kfp', 'kfp'),
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

## API Enablement

In [4]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [5]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

Inputs

In [6]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [7]:
REGION = 'us-central1'
EXPERIMENT = 'pipeline-notifications'
SERIES = 'mlops'

# gcs bucket
GCS_BUCKET = PROJECT_ID

Packages

In [8]:
import os
import time
import importlib
from google.cloud import aiplatform
import kfp
from typing import NamedTuple

In [9]:
kfp.__version__

'2.7.0'

In [10]:
aiplatform.__version__

'1.51.0'

Clients

In [11]:
# vertex ai clients
aiplatform.init(project = PROJECT_ID, location = REGION)

parameters:

In [12]:
DIR = f"temp/{SERIES}-{EXPERIMENT}"

In [13]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

environment:

In [14]:
if not os.path.exists(DIR):
    os.makedirs(DIR)

---
## Example Components

Components that:
- generate coin flips with `flip_coin`
    - by default it returns flip of a single coin as 'H' or 'T'
    - optional input parameter of `num_coins` can be set to number of coins to retrive a string of flips, like 2 => 'HT'
- generate dice rolls with `roll_dice`
    - by default it returns the face number [1, 6] from a single die roll
    - optionn input parameter of `num_dice` an be set to number of dice to retrieve a sum of rolls, like 2 => [2, 12]

In [15]:
@kfp.dsl.component(base_image = 'python:3.10')
def flip_coins(num_coins: int = 1) -> str:
    import random
    flipmap = ['T', 'H']
    flips = [flipmap[random.randint(0, 1)] for n in range(num_coins)]
    return ''.join(flips)

@kfp.dsl.component(base_image = 'python:3.10')
def roll_dice(num_dice: int = 1) -> int:
    import random
    result = sum([random.randint(1,6) for n in range(num_dice)])
    return result

---
## Pre-Built Notifications Components

- https://cloud.google.com/vertex-ai/docs/pipelines/email-notifications
- https://cloud.google.com/vertex-ai/docs/pipelines/gcpc-list#emailnotification_components
- https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-2.14.1/api/v1/vertex_notification_email.html#v1.vertex_notification_email.VertexNotificationEmailOp

In [16]:
from google_cloud_pipeline_components.v1.vertex_notification_email import VertexNotificationEmailOp

In [17]:
@kfp.dsl.pipeline(
    name = f'{SERIES}-{EXPERIMENT}-notify',
    pipeline_root = f'gs://{GCS_BUCKET}/{SERIES}/{EXPERIMENT}/pipeline_root'
)
def notify_pipeline():
    
    task_1 = roll_dice()
    notify_complete = VertexNotificationEmailOp(recipients = ['statmike@google.com'])
    
    with kfp.dsl.ExitHandler(exit_task = notify_complete):
        task_2 = flip_coins(num_coins = task_1.output)

### Compile Pipeline

In [18]:
kfp.compiler.Compiler().compile(
    pipeline_func = notify_pipeline,
    package_path = f'{DIR}/{SERIES}-{EXPERIMENT}-notify.yaml'
)

### Create Pipeline Job (With Vertex AI SDK)

The compiled pipeline file can be submitted for running with the console or the SDK (shown here).  Check out the details in the documentation [here](https://cloud.google.com/vertex-ai/docs/pipelines/run-pipeline#create_a_pipeline_run) for an overview with the console.


In [19]:
pipeline_job = aiplatform.PipelineJob(
    display_name = f"{SERIES}-{EXPERIMENT}-notify",
    template_path = f"{DIR}/{SERIES}-{EXPERIMENT}-notify.yaml",
    pipeline_root = f'gs://{GCS_BUCKET}/{SERIES}/{EXPERIMENT}/pipeline_root',
    enable_caching = None # True (enabled), False (disable), None (defer to component level caching) 
)

### Submit Pipeline Job (On Vertex AI Pipelines)

In [20]:
response = pipeline_job.submit(
    service_account = SERVICE_ACCOUNT
)

Creating PipelineJob
PipelineJob created. Resource name: projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-notifications-notify-20240613182955
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-notifications-notify-20240613182955')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/mlops-pipeline-notifications-notify-20240613182955?project=1026793852137


In [21]:
print(f'The Dashboard can be viewed here:\n{pipeline_job._dashboard_uri()}')

The Dashboard can be viewed here:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/mlops-pipeline-notifications-notify-20240613182955?project=1026793852137


In [22]:
pipeline_job.wait()

PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-notifications-notify-20240613182955 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-notifications-notify-20240613182955 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-notifications-notify-20240613182955 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob run completed. Resource name: projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-notifications-notify-20240613182955


## Get Pipelines Status

- https://kubeflow-pipelines.readthedocs.io/en/stable/source/dsl.html#kfp.dsl.PipelineTaskFinalStatus
- https://kubeflow-pipelines.readthedocs.io/en/stable/source/dsl.html#kfp.dsl.PipelineTaskFinalStatus

In [23]:
@kfp.dsl.component(base_image = 'python:3.10')
def exit_op(status: kfp.dsl.PipelineTaskFinalStatus) -> dict:
    response = status.__dict__
    return response

In [24]:
@kfp.dsl.component(base_image = 'python:3.10')
def force_fail():
    import sys
    sys.exit(1)

In [25]:
@kfp.dsl.pipeline(
    name = f'{SERIES}-{EXPERIMENT}-status',
    pipeline_root = f'gs://{GCS_BUCKET}/{SERIES}/{EXPERIMENT}/pipeline_root'
)
def status_pipeline():
    
    task_status = exit_op()
    with kfp.dsl.ExitHandler(exit_task = task_status):
        task_1 = roll_dice()
        task_fail = force_fail().after(task_1)
        task_2 = flip_coins(num_coins = task_1.output)

### Compile Pipeline

In [26]:
kfp.compiler.Compiler().compile(
    pipeline_func = status_pipeline,
    package_path = f'{DIR}/{SERIES}-{EXPERIMENT}-status.yaml'
)

### Create Pipeline Job (With Vertex AI SDK)

The compiled pipeline file can be submitted for running with the console or the SDK (shown here).  Check out the details in the documentation [here](https://cloud.google.com/vertex-ai/docs/pipelines/run-pipeline#create_a_pipeline_run) for an overview with the console.


In [27]:
pipeline_job = aiplatform.PipelineJob(
    display_name = f"{SERIES}-{EXPERIMENT}-status",
    template_path = f"{DIR}/{SERIES}-{EXPERIMENT}-status.yaml",
    pipeline_root = f'gs://{GCS_BUCKET}/{SERIES}/{EXPERIMENT}/pipeline_root',
    enable_caching = None # True (enabled), False (disable), None (defer to component level caching) 
)

### Submit Pipeline Job (On Vertex AI Pipelines)

In [28]:
response = pipeline_job.submit(
    service_account = SERVICE_ACCOUNT
)

Creating PipelineJob
PipelineJob created. Resource name: projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-notifications-status-20240613183108
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-notifications-status-20240613183108')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/mlops-pipeline-notifications-status-20240613183108?project=1026793852137


In [29]:
print(f'The Dashboard can be viewed here:\n{pipeline_job._dashboard_uri()}')

The Dashboard can be viewed here:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/mlops-pipeline-notifications-status-20240613183108?project=1026793852137


In [30]:
try:
    pipeline_job.wait()
except Exception as err:
    print(f"{type(err).__name__} was raised: {err}")

PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-notifications-status-20240613183108 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-notifications-status-20240613183108 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-notifications-status-20240613183108 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/mlops-pipeline-notifications-status-20240613183108 current state:
PipelineState.PIPELINE_STATE_RUNNING
RuntimeError was raised: Job failed with:
code: 9
message: "The DAG failed because some tasks failed. The failed tasks are: [exit-handler-1].; Job (project_id = statmike-mlops-349915, job_id = 1052799151147843584) is failed due to the above error.; Failed to handle the job: {project_number = 1026793852137, 

### Retrieve all runs to dataframe:
- SDK Refrence: [`aiplatform.get_pipeline_df`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_get_pipeline_df)

In [31]:
aiplatform.get_pipeline_df(pipeline = f'{SERIES}-{EXPERIMENT}-status')

,pipeline_name,run_name,param.vmlmd_lineage_integration
0,mlops-pipeline-notifications-status,mlops-pipeline-notifications-status-2024061318...,{'pipeline_run_component': {'location_id': 'us...
1,mlops-pipeline-notifications-status,mlops-pipeline-notifications-status-2024061318...,{'pipeline_run_component': {'location_id': 'us...
2,mlops-pipeline-notifications-status,mlops-pipeline-notifications-status-2024061317...,{'pipeline_run_component': {'parent_task_names...
3,mlops-pipeline-notifications-status,mlops-pipeline-notifications-status-2024061317...,{'pipeline_run_component': {'task_name': 'mlop...


In [32]:
for task in pipeline_job.task_details:
    print(task.task_name, task.state)

flip-coins State.SKIPPED
mlops-pipeline-notifications-status-20240613183108 State.FAILED
roll-dice State.SKIPPED
exit-op State.SUCCEEDED
force-fail State.FAILED
exit-handler-1 State.FAILED


---
## Use Pipeline Status To Conditionally Send Email Notifications